In [1]:
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from sklearn.linear_model import ElasticNet
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.metrics import mean_squared_error, r2_score
from itertools import chain
from matplotlib import pyplot as plt
from sklearn.pipeline import make_pipeline
from sklearn.inspection import permutation_importance
import numpy as np
import altair as alt
from scipy.stats import f_oneway

/home/saurabhjhanjee/anaconda3/envs/applied-data-science1/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
data = pd.read_parquet("../data/curated/train/Airport Dropoffs_2021_train")
y = data["count"]
AIRPORT_CODE_MAP = {132: "JFK", 1: "EWR", 138: "LGA"}

# ANOVA
Test for relevance of Day

In [3]:
import statsmodels.api as sm
from statsmodels.formula.api import ols

model = ols('count ~ C(Day)', data=data).fit()

anova_table = sm.stats.anova_lm(model, typ=2)
print(anova_table.to_latex())

\begin{tabular}{lrrrr}
\toprule
{} &        sum\_sq &       df &          F &        PR(>F) \\
\midrule
C(Day)   &  6.272285e+06 &      6.0 &  38.286814 &  2.718924e-46 \\
Residual &  3.035925e+08 &  11119.0 &        NaN &           NaN \\
\bottomrule
\end{tabular}



/tmp/ipykernel_50323/2035403004.py:7: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(anova_table.to_latex())


Test for relevance of Facility

In [4]:
model = ols('count ~ C(Facility)', data=data).fit()
anova_table = sm.stats.anova_lm(model, typ=2)
print(anova_table.to_latex())

\begin{tabular}{lrrrr}
\toprule
{} &        sum\_sq &       df &            F &  PR(>F) \\
\midrule
C(Facility) &  6.338480e+07 &      2.0 &  1430.195648 &     0.0 \\
Residual    &  2.464800e+08 &  11123.0 &          NaN &     NaN \\
\bottomrule
\end{tabular}



/tmp/ipykernel_50323/3196118863.py:3: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(anova_table.to_latex())


Test for interaction of Facility and Day

In [5]:
model = ols('count ~ C(Facility) + C(Day) + C(Facility):C(Day)', data=data).fit()
anova_table = sm.stats.anova_lm(model, typ=2)
print(anova_table.to_latex())


\begin{tabular}{lrrrr}
\toprule
{} &        sum\_sq &       df &            F &        PR(>F) \\
\midrule
C(Facility)        &  6.385527e+07 &      2.0 &  1492.334127 &  0.000000e+00 \\
C(Day)             &  6.742758e+06 &      6.0 &    52.527365 &  3.904188e-64 \\
C(Facility):C(Day) &  2.152066e+06 &     12.0 &     8.382503 &  5.128350e-16 \\
Residual           &  2.375851e+08 &  11105.0 &          NaN &           NaN \\
\bottomrule
\end{tabular}



/tmp/ipykernel_50323/4186352112.py:3: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(anova_table.to_latex())


In [6]:
df = data.copy()
df.columns = df.columns.str.replace(' ', '_')
model = ols('count ~ Arrivals_For_Metric_Computation', data=df).fit()
model2 = ols('count ~ Departures_For_Metric_Computation + Arrivals_For_Metric_Computation', data=df).fit()

anova_table = sm.stats.anova_lm(model, model2, typ=1)
print(anova_table.to_latex())


\begin{tabular}{lrrrrrr}
\toprule
{} &  df\_resid &           ssr &  df\_diff &       ss\_diff &           F &        Pr(>F) \\
\midrule
0 &   11124.0 &  3.037303e+08 &      0.0 &           NaN &         NaN &           NaN \\
1 &   11123.0 &  2.978136e+08 &      1.0 &  5.916760e+06 &  220.984281 &  1.645919e-49 \\
\bottomrule
\end{tabular}



/tmp/ipykernel_50323/3087735198.py:7: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(anova_table.to_latex())


Plot correlations between time shifted vars and count.

In [7]:


def plot_corr(x, y, title):
    base = alt.Chart(corrMatrix1).encode(
        x=x,
        y=y,
    ).properties(
        width=alt.Step(100),
        height=alt.Step(100),
        title=title
    )

    rects = base.mark_rect().encode(
        color=alt.Color('correlation', scale=alt.Scale(domain=[-1,1]))
    )

    text = base.mark_text(
        size=30
    ).encode(
        text=alt.Text('correlation', format=".2f"),
        color=alt.condition(
            "datum.correlation > 0.5",
            alt.value('white'),
            alt.value('black')
        )
    )

    return rects + text


In [8]:
corrMatrix = data.drop(["Facility", "Day", "Hour"], axis=1).corr().loc["count", :].reset_index().melt('index')
corrMatrix.columns = ['Possible Features', 'Response', 'correlation']
def rename_columns(value, ride_type):
    if value == "count": return ride_type + " Count"
    if value[-1].isnumeric(): return value[-2:]
    else: return "0"

corrMatrix1 = corrMatrix.copy()
corrMatrix1 = corrMatrix1[corrMatrix1['Possible Features'].str.contains("Arrivals")]
corrMatrix1['Possible Features'] = corrMatrix1['Possible Features'].apply(lambda x: rename_columns(x, "Dropoffs"))


plot_corr("Possible Features", "Response", "Airport Dropoff Count vs Temporally Shifted Plane Arrivals")

alt.LayerChart(...)

In [9]:
corrMatrix1 = corrMatrix.copy()
corrMatrix1 = corrMatrix1[corrMatrix1['Possible Features'].str.contains("Departures")]
corrMatrix1['Possible Features'] = corrMatrix1['Possible Features'].apply(lambda x: rename_columns(x, "Dropoffs"))

plot_corr("Possible Features", "Response", "Airport Dropoff Count vs Temporally Shifted Plane Departures")

alt.LayerChart(...)

In [10]:
pickup_data = pd.read_parquet("../data/curated/train/Airport Pickups_2021_train")
pickup_y = pickup_data["count"]


In [11]:
pickupCorrMatrix = pickup_data.drop(["Facility", "Day", "Hour"], axis=1).corr().loc["count", :].reset_index().melt('index')
pickupCorrMatrix.columns = ['Possible Features', 'Response', 'correlation']
corrMatrix1 = pickupCorrMatrix.copy()
corrMatrix1 = corrMatrix1[corrMatrix1['Possible Features'].str.contains("Arrivals")]
corrMatrix1['Possible Features'] = corrMatrix1['Possible Features'].apply(lambda x: rename_columns(x, "Pickups"))

plot_corr("Possible Features", "Response", "Airport Pickup Count vs Temporally Shifted Plane Arrivals")

alt.LayerChart(...)

In [12]:
corrMatrix1 = pickupCorrMatrix.copy()
corrMatrix1 = corrMatrix1[corrMatrix1['Possible Features'].str.contains("Departures")]
corrMatrix1['Possible Features'] = corrMatrix1['Possible Features'].apply(lambda x: rename_columns(x, "Pickups"))

plot_corr("Possible Features", "Response", "Airport Pickup Count vs Temporally Shifted Plane Departures")

alt.LayerChart(...)

In [13]:
feature_list = ["Day", "Departures For Metric Computation", "Arrivals For Metric Computation", "Facility"]
feature_list = []
hour_intervals = range(1, 6)
for interval in hour_intervals:
    feature_list.append(f"Departures For Metric Computation+{interval}")
    # feature_list.append(f"Departures For Metric Computation-{interval}")
    feature_list.append(f"Arrivals For Metric Computation-{interval}")
    # feature_list.append(f"Arrivals For Metric Computation-{interval}")


Plot pairs correlation between time shifted vars to determine any collinearity

In [14]:
# feature_list = ["Departures For Metric Computation", "Arrivals For Metric Computation"]
corrMatrix = data[[*feature_list]].corr().reset_index().melt('index')
corrMatrix.columns = ['Possible Features', 'Response', 'correlation']


base = alt.Chart(corrMatrix).encode(
    x='Possible Features',
    y='Response',
).properties(
    width=alt.Step(100),
    height=alt.Step(100)
)

rects = base.mark_rect().encode(
    color=alt.Color('correlation', scale=alt.Scale(domain=[0,1]))
)

text = base.mark_text(
    size=30
).encode(
    text=alt.Text('correlation', format=".2f"),
    color=alt.condition(
        "datum.correlation > 0.5",
        alt.value('white'),
        alt.value('black')
    )
)

rects + text

alt.LayerChart(...)

Test stuff on our model! This is not really used for much.

In [15]:

X = pd.get_dummies(pickup_data[feature_list], columns=["Day", "Facility", ])
pipeline = make_pipeline(PolynomialFeatures(interaction_only=True, include_bias = False), StandardScaler(), ElasticNet())

X_train,X_test,y_train,y_test=train_test_split(X,pickup_y)

KeyError: "None of [Index(['Day', 'Facility'], dtype='object')] are in the [columns]"

In [ ]:
params =  {"elasticnet__alpha": [.1, 1],
                      "elasticnet__l1_ratio": [.7, 1]}

gs = GridSearchCV(pipeline, params, scoring="r2")

gs.fit(X_train, y_train)

X_test = pd.read_parquet("../data/curated/test/Airport Pickups_2022_test")
y_test = X_test["count"]
X_test = pd.get_dummies(X_test[feature_list], columns=["Day", "Facility"])
pred = gs.best_estimator_.predict(X_test)
r2_score(y_test, pred)

In [ ]:
gs.best_params_

In [ ]:
gs.best_score_